# Naive one-shot LLM vs ReAct with tools — comparison

In [ ]:
import os, textwrap
from openai import OpenAI

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
MODEL_NAME = "gpt-4o-mini"

user_query = (
    "I am traveling to Bangalore for a 3-day business trip near the main tech park. "
    "Recommend a good business hotel and tell me what to pack based on the current weather. "
    "I prefer not to spend more than 5500 INR per night."
)
print("Using model:", MODEL_NAME)
print("User query:\n", user_query)

## 1. Naive one-shot answer

In [ ]:
def naive_llm_answer(query: str) -> str:
    resp = client.chat.completions.create(
        model=MODEL_NAME,
        messages=[
            {"role": "system", "content": "You are a helpful travel assistant."},
            {"role": "user", "content": query},
        ],
        temperature=0.7,
    )
    return resp.choices[0].message.content

naive_answer = naive_llm_answer(user_query)
print("Naive LLM answer:\n")
print(naive_answer)

## 2. ReAct agent answer (reusing simplified logic)

In [ ]:
from dataclasses import dataclass
from typing import List, Dict, Callable, Optional
import requests

@dataclass
class ToolResult:
    name: str
    input: str
    output: str

def hotel_search_bangalore(query: str) -> str:
    hotels = [
        {
            "name": "Orion Business Hotel",
            "area": "Whitefield",
            "distance_km_to_office": 1.2,
            "price_band_inr": "4500-5500",
            "notes": "Walkable to tech park, good Wi-Fi, quiet, popular with business travelers."
        },
        {
            "name": "Skyline Executive Suites",
            "area": "Indiranagar",
            "distance_km_to_office": 8.0,
            "price_band_inr": "5000-6500",
            "notes": "Great restaurants nearby, slightly longer commute in traffic."
        },
        {
            "name": "Airport Link Business Hotel",
            "area": "Near Airport",
            "distance_km_to_office": 30.0,
            "price_band_inr": "4000-5200",
            "notes": "Good for late-night arrivals, but long commute to office area."
        },
    ]
    lines = ["Bangalore business hotels (internal catalog):", ""]
    for h in hotels:
        lines.append(
            f"- {h['name']} ({h['area']}): "
            f"{h['distance_km_to_office']}km to office, "
            f"{h['price_band_inr']} INR/night. Notes: {h['notes']}"
        )
    lines.append("")
    lines.append("Choose one that balances commute time and comfort based on traveler role and arrival time.")
    return "\n".join(lines)

def weather_snapshot(city: str) -> str:
    try:
        if city.lower().startswith("bangalore") or city.lower().startswith("bengaluru"):
            lat, lon = 12.9716, 77.5946
        else:
            lat, lon = 12.9716, 77.5946
        resp = requests.get(
            "https://api.open-meteo.com/v1/forecast",
            params={"latitude": lat, "longitude": lon, "current_weather": True},
            timeout=10,
        )
        if resp.status_code != 200:
            return f"[weather] API error {resp.status_code}: {resp.text[:200]}"
        data = resp.json()
        cw = data.get("current_weather", {})
        temp_c = cw.get("temperature")
        wind = cw.get("windspeed")
        cond_code = cw.get("weathercode")
        return (
            f"Current weather for {city}: {temp_c}°C, wind {wind} km/h, "
            f"weathercode {cond_code} (see open-meteo codes). "
            f"Evenings can feel cooler; pack a light jacket."
        )
    except Exception as e:
        return f"[weather] fallback: could not retrieve live data ({e}). Assume warm but variable; pack breathable clothes and a light jacket."

TOOLS: Dict[str, Callable[[str], str]] = {
    "hotel_search_bangalore": hotel_search_bangalore,
    "weather_snapshot": weather_snapshot,
}

TOOL_DESCRIPTIONS = """
You have access to the following tools:

1. hotel_search_bangalore(query: str)
2. weather_snapshot(city: str)
"""

SYSTEM_PROMPT = f"""
You are a ReAct-style travel agent. Follow this format:

Thought: <your reasoning>
Action: <hotel_search_bangalore or weather_snapshot>
Action Input: <a single-line argument>

... repeat Thought -> Action -> Observation as needed ...

When done, respond with:
Final Answer: <your complete answer>
"""

def call_llm(messages: List[Dict[str, str]]) -> str:
    resp = client.chat.completions.create(
        model=MODEL_NAME,
        messages=messages,
        temperature=0.3,
    )
    return resp.choices[0].message.content

def parse_react_step(text: str):
    lines = [l.strip() for l in text.splitlines() if l.strip()]
    for line in lines:
        if line.startswith("Final Answer:"):
            return {"type": "final", "answer": line[len("Final Answer:"):].strip()}
    tool = None
    tool_input = None
    for line in lines:
        if line.startswith("Action:"):
            tool = line[len("Action:"):].strip()
        elif line.startswith("Action Input:"):
            tool_input = line[len("Action Input:"):].strip().strip('"')
    if tool and tool_input is not None:
        return {"type": "action", "tool": tool, "tool_input": tool_input}
    return {"type": "unknown", "raw": text}

def run_react_agent(query: str, max_steps: int = 6):
    messages = [
        {"role": "system", "content": SYSTEM_PROMPT + "\n" + TOOL_DESCRIPTIONS},
        {"role": "user", "content": query},
    ]
    trace: List[ToolResult] = []
    final_answer = None
    for step in range(1, max_steps + 1):
        print(f"\n=== ReAct Step {step} ===")
        out = call_llm(messages)
        print("LLM raw output:\n", out)
        parsed = parse_react_step(out)
        if parsed["type"] == "final":
            final_answer = parsed["answer"]
            print("[ReAct agent produced Final Answer]")
            break
        if parsed["type"] == "action":
            tool_name = parsed["tool"]
            tool_input = parsed["tool_input"]
            if tool_name in TOOLS:
                obs = TOOLS[tool_name](tool_input)
            else:
                obs = f"[error] unknown tool: {tool_name}"
            trace.append(ToolResult(name=tool_name, input=tool_input, output=obs))
            messages.append({"role": "assistant", "content": out})
            messages.append({"role": "user", "content": f"Observation: {obs}"})
            continue
        messages.append({"role": "assistant", "content": out})
    return {"trace": trace, "final_answer": final_answer}

react_result = run_react_agent(user_query, max_steps=6)
print("\nReAct FINAL ANSWER:\n", react_result["final_answer"])

## 3. Side-by-side comparison

In [ ]:
print("===== Naive one-shot answer =====\n")
print(textwrap.indent(naive_answer, prefix="  "))

print("\n\n===== ReAct tool-grounded answer =====\n")
print(textwrap.indent(react_result["final_answer"], prefix="  "))

## 4. Inspect ReAct tool calls

In [ ]:
for i, tr in enumerate(react_result["trace"], start=1):
    print(f"\n---------- ReAct Tool Call {i} ----------")
    print(f"Tool: {tr.name}")
    print(f"Input: {tr.input}")
    print("Output:")
    print(textwrap.indent(tr.output, prefix="  "))